In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install langchain


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.5 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 12.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 71.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 36.8 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-r6c0of_e/overlay/local/bin
    changing mode of /tmp/pip-build-env-r6c0of_e/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-r6c0of_e/overlay/

In [59]:
%clear

In [23]:
!pip install tqdm

In [24]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from tqdm import tqdm

In [3]:
model_name_or_path = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_K_M.bin" # the model is in bin format

In [4]:
from huggingface_hub import hf_hub_download


In [5]:
from llama_cpp import Llama

In [6]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)


llama-2-7b-chat.ggmlv3.q4_K_M.bin:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

In [7]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [77]:
# GPU
lcpp_llm = None
lcpp_llm = LlamaCpp(
    model_path=model_path,
    n_ctx=4096,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    callback_manager=callback_manager,
    #verbose=True
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [78]:
# See the number of layers in GPU
lcpp_llm.client.params.n_gpu_layers

32

In [79]:
template ="""[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a concise summary of the text, return your responses with 3 sentences that cover the key points of the text. Your answer should only include the 3 sentences.
```{text}```

SUMMARY:[/INST]
"""

In [80]:
prompt = PromptTemplate(template=template, input_variables=["text"])


In [81]:
llmchain = LLMChain(prompt=prompt,llm=lcpp_llm)

In [82]:
from pathlib import Path
import pandas as pd
import numpy as np

In [83]:
papers_dataset_path=Path("data/papers_dataset.csv")
papers_dataset = pd.read_csv(papers_dataset_path)

In [84]:
extracted_texts_path=Path("data/preprocessed-text")

In [85]:
with tqdm(total=len(papers_dataset)) as progress:
  for i in papers_dataset.index:

      #retrieve extracted paper
      paper_name = papers_dataset.iloc[i]["paper"]
      print(paper_name)

      with open(Path(extracted_texts_path,paper_name+'.txt')) as f:
          paper_content = f.read()
      llm_response = llmchain.run(paper_content[:4000])
      print("done")
      papers_dataset.at[i,"summary"] = llm_response
      progress.update()


  0%|          | 0/14 [00:00<?, ?it/s]

82_2020_217
Sure, I'd be happy to help! Here are three key points from the text:

1. In vitro transcribed mRNA has shown promise as a vaccine platform due to its fast development process and ability to encode various proteins.
2. Different administration routes for mRNA vaccines, such as intramuscular, intradermal, subcutaneous, and intravenous injection, can impact the safety and efficacy of vaccination.
3. Researchers have investigated various delivery strategies for mRNA vaccines, including the use of lipid-based nanoparticles and other carriers, to improve cellular uptake and delivery of the vaccine.

  7%|▋         | 1/14 [00:50<10:56, 50.48s/it]

done
biomedicines-11-00308-v2


Llama.generate: prefix-match hit


...

done
Safety and Efficacy of the BNT162b2 mRNA Covid-19 Vaccine


Llama.generate: prefix-match hit


The article discusses the results of a phase 2/3 trial evaluating the safety and efficacy of BNT162b2, a COVID-19 vaccine candidate, in preventing the disease. The trial included adults aged 16 or older with either stable chronic medical conditions or were healthy, and excluded those with a history of COVID-19, treatment with immunosuppressive therapy, or diagnosis of an immunocompromising condition. Participants were randomly assigned to receive either BNT162b2 or placebo, and safety data were collected for 14 weeks after the second dose. The primary endpoints of the trial were solicited adverse events and unsolicited serious adverse events through 6 months after the second dose. The results showed that BNT162b2 was efficacious in preventing COVID-19 with onset at least 7 days after the second dose, and the vaccine was well-tolerated with few solicited adverse events reported.

100%|██████████| 14/14 [11:59<00:00, 51.42s/it]

done


In [86]:
papers_dataset

,paper,summary,abstract
0,82_2020_217,"Sure, I'd be happy to help! Here are three key...",mRNA vaccines have become a versatile technolo...
1,biomedicines-11-00308-v2,Sure! Here are three sentences that summarize ...,This review provides an overview covering mRNA...
2,Efficacy and Safety of the mRNA-1273 SARS-CoV-...,"Sure, here is a concise summary of the text in...",Vaccines are needed to prevent coronavirus dis...
3,Emerging Frontiers in Drug Delivery,The text discusses drug delivery systems (DDS)...,Medicine relies on the use of pharmacologicall...
4,mRNA vaccines — a new era(1),"Sure, here is a concise summary of the text in...",Vaccines prevent many millions of illnesses an...
5,mRNA vaccines — a new era,Here is a summary of the text in three sentenc...,Vaccines prevent many millions of illnesses an...
6,nanomaterials-10-00364-v2,Gene therapy involves using genetic material t...,The use of messenger RNA (mRNA) in gene therap...
7,pharmaceutics-12-00102-v2,Vaccination has significantly improved global ...,"In the past few years, there has been increasi..."
8,PMC8198544,Synthetic mRNA has gained significant attentio...,Synthetic modified messenger RNA (mRNA) has ma...
9,PMC8325057,The text discusses the recent approval of the ...,The recent approval of messenger RNA (mRNA)-ba...


In [87]:
csv_path=Path("updated_papers_dataset.csv")
papers_dataset.to_csv(csv_path,index=False)